In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Making Dataset

num_sample = 250 

# XOR Problem 
data0 = np.random.randn(num_sample, 2) + (2,2)
data1 = np.random.randn(num_sample, 2) + (-2,-2)
data2 = np.random.randn(num_sample, 2) + (2,-2)
data3 = np.random.randn(num_sample, 2) + (-2,2)

# labeling 
data0 = np.hstack([data0, np.zeros((num_sample, 1), dtype=float)])
data1 = np.hstack([data1, np.zeros((num_sample, 1), dtype=float)])
data2 = np.hstack([data2, np.ones((num_sample, 1), dtype=float)])
data3 = np.hstack([data3, np.ones((num_sample, 1), dtype=float)])

# make into single dataset
data = np.vstack([data0, data1, data2, data3])
data = torch.tensor(data, dtype=torch.float32)

print(data.shape)

torch.Size([1000, 3])


In [4]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(2,3)
        self.linear2 = nn.Linear(3,3)
        self.linear3 = nn.Linear(3,1)
        
    def forward(self, x):
        x = F.leaky_relu(self.linear1(x)) # hidden
        x = F.tanh(self.linear2(x)) # hidden 
        x = F.sigmoid(self.linear3(x)) # output
        # x = self.linear3(x) # logit : inverse of sigmoid(softmax)
        return x

In [5]:
model = LogisticRegression()
loss = nn.BCELoss() # need sigmoid activation
# loss = nn.BCEWithLogitsLoss : do not need sigmoid, logit만 있으면 됨 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [6]:
EPOCHS = 200
loss_lst = list()

In [21]:
for i in range(EPOCHS):
    shuffled_dataset = np.random.permutation(data)
    shuffled_dataset = torch.tensor(shuffled_dataset)
    total_loss = 0
    cnt = 0
    
    for j in range(len(shuffled_dataset)):
        x = shuffled_dataset[j][:-1]
        y = shuffled_dataset[j][-1]
        
        y = y.reshape([1]) # torch.Size([1])
        pred = model.forward(x) # output = P(y=1|x)
        # logit = mpdel.forward(x)
        # prob = F.sigmoid(logit)
        
        cost = loss(pred, y) # cal loss
        # cost = loss(logit, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        total_loss += cost.item()
        
        pred = torch.round(pred).item()
        cnt += (pred == y.item())
        
    acc = cnt / len(shuffled_dataset)
    loss_lst.append(total_loss)
    
    if i % 10 ==0:
        print(f"Epoch {i}: {total_loss} / {acc}")

Epoch 0: 257.6317257359624 / 0.919
Epoch 10: 124.21806702786125 / 0.955
Epoch 20: 122.2580069063697 / 0.958
Epoch 30: 120.73169162636623 / 0.958
Epoch 40: 117.7627717081923 / 0.958
Epoch 50: 117.65909820585512 / 0.961
Epoch 60: 115.21672062086873 / 0.964
Epoch 70: 110.96346937050112 / 0.963
Epoch 80: 110.51253686391283 / 0.959
Epoch 90: 109.14128299325239 / 0.966
Epoch 100: 103.72953682625666 / 0.967
Epoch 110: 110.09194405714516 / 0.96
Epoch 120: 104.2374648269033 / 0.964
Epoch 130: 111.34838612168096 / 0.964
Epoch 140: 103.63615949382074 / 0.963
Epoch 150: 102.4415699786041 / 0.958
Epoch 160: 106.58613431686535 / 0.96
Epoch 170: 98.74705228558742 / 0.963
Epoch 180: 100.82493243878707 / 0.964
Epoch 190: 97.07426498946734 / 0.965
